In [195]:
import requests
import json
import urllib.request
from PIL import Image
import time
import os
import glob
import random
import shutil

headers = {
  'Content-Type': 'application/json',
  'api_key': 'dc6002bd-c916-40ed-acd1-fd507190b468'
}


permanent_json_paths = glob.glob('/content/permanent_jsons/*')

for permanent_json in permanent_json_paths:
  permanent_json_name = os.path.basename(permanent_json).split('.')[0]
  shutil.copyfile(permanent_json, f'/content/inference_jsons/{permanent_json_name}.json')

In [165]:
get_models_url = "https://api.lightsketch.ai/v1/models?model_type=sd1.5&ownership=public"
get_sampler_url = "https://api.lightsketch.ai/v1/samplers"

model_response = requests.request("GET", get_models_url, headers=headers)
sampler_response = requests.request("GET", get_sampler_url, headers=headers)

model_list = model_response.json()["items"]
sampler_list = sampler_response.json()

model_ids = []
for model in model_list:
  model_ids += [model["id"]]

sampler_names = []
for sampler in sampler_list:
  sampler_names += [sampler["name"]]

In [166]:
for model in model_ids:
  sampler = random.choice(sampler_names)

  test_json = json.dumps({
    "model_id": model,
    "prompt": "a man",
    "seed": 8457691,
    "sampler_name": sampler,
    "num_generations": 1,
    "cfg_scale": 7,
    "width": 512,
    "height": 512,
    "steps": 20
  })

  with open(f"/content/inference_jsons/{model}_{sampler}.json", "w") as outfile:
    outfile.write(test_json)


In [ ]:
def Average(lst):
    return sum(lst) / len(lst)

json_paths = glob.glob('/content/inference_jsons/*')
time_counter_list = []

post_url = 'https://api.lightsketch.ai/v1/generation/text-to-image'

for json_file_path in json_paths:
  json_name = os.path.basename(json_file_path).split('.')[0]

  parameters = json.load(open(json_file_path))
  payload = json.dumps(parameters)

  post_response = requests.request("POST", post_url, headers=headers, data=payload)
  try:
    id = post_response.json()["id"]
  except:
    print(f'\n POST request for {json_name} failed')
    print(post_response.text)
    print('\n')
    continue

  get_url = "https://api.lightsketch.ai/v1/generation/text-to-image/" + id
  status = 'generating'
  time_counter = 0

  while status in ['generating','queued'] and time_counter < 60:
    time.sleep(1)
    get_response = requests.request("GET", get_url, headers=headers)
    status = get_response.json()["status"]
    time_counter += 1

  if time_counter >= 60:
    print(f'\n generation for {json_name} took too long, skipping')
    print(response.text)
    print('\n')
    continue

  samples = get_response.json()["sample_images"]

  if json_name.split('_')[0] != "perma":
    time_counter_list += [time_counter]

  print(f"{json_name}: {status} in less than {time_counter}sec")

  urllib.request.urlretrieve(samples[0], "sample.jpeg")

  img = Image.open("sample.jpeg")
  img.save(f"/content/samples/{json_name}.jpg")

average_generation_time = Average(time_counter_list)
print(f'average generation time: {average_generation_time}')


sd15L5lQDZWa_DPM++ 2M: success in less than 6sec
perma_large_image
perma_large_image: success in less than 16sec
sd156FRJrxka_DPM++ 2S a: success in less than 4sec
perma_cfg
perma_cfg: success in less than 7sec
permanent_jsons: success in less than 13sec

 POST request for sd15BRoNQOoa_DPM++ 3M SDE Karras failed
{"detail":[{"type":"literal_error","loc":["body","sampler_name"],"msg":"Input should be 'Euler a', 'Euler', 'LMS', 'Heun', 'DPM2', 'DPM2 a', 'DPM++ 2S a', 'DPM++ 2M', 'DPM++ SDE', 'DPM++ 2M SDE', 'DPM fast', 'DPM adaptive', 'LMS Karras', 'DPM2 Karras', 'DPM2 a Karras', 'DPM++ 2S a Karras', 'DPM++ 2M Karras', 'DPM++ SDE Karras', 'DPM++ 2M SDE Karras', 'DDIM', 'PLMS' or 'UniPC'","input":"DPM++ 3M SDE Karras","ctx":{"expected":"'Euler a', 'Euler', 'LMS', 'Heun', 'DPM2', 'DPM2 a', 'DPM++ 2S a', 'DPM++ 2M', 'DPM++ SDE', 'DPM++ 2M SDE', 'DPM fast', 'DPM adaptive', 'LMS Karras', 'DPM2 Karras', 'DPM2 a Karras', 'DPM++ 2S a Karras', 'DPM++ 2M Karras', 'DPM++ SDE Karras', 'DPM++ 2M SDE K